In [4]:
from IPython.display import display, Image
import xml.etree.ElementTree as ET
import requests
#import pythoncyc as pc
#from HTMLParser import HTMLParser
#from lxml import html
#import biopython

# this function finds all pathways associated with the molecule of interest. needs work on front end to 
# allow user to define G.O. function associated with molecule.
class organismBreadBoard:
    def __init__(self, name):
        self.name = name
    def pathwayFromCompound(self, compound):
        self.compound = compound
        pathWays = {}
    #request pathway xml from ecocyc server
        resp = requests.get("http://ecocyc.org/apixml?fn=pathways-of-compound&id=ECOLI:{}".format(self.compound))
    #builds element tree from the response xml text file
        root = ET.fromstring(resp.text)
    #pulls all elements associated with Pathway tag
        pathways = root.findall("./Pathway")
    #pulls all pathway IDs
        for pathway in pathways:
            pathID = pathway.attrib['frameid']
            pathWays[pathID] = pathway.findall("common-name")[0].text
        #print (pathWays)
    #return dictionary with pathway ID: common name as key:value pairs
    #print pathWays
        return pathWays

# this function pulls all geneIDs associated with the pathwayID passed into it
    def genesFromPathway(self, pathwayID):
        resp  = requests.get("http://ecocyc.org/apixml?fn=genes-of-pathway&id=ECOLI:{}&detail=full".format(pathwayID))
        root = ET.fromstring(resp.text)
    #print(resp.text)
        genes = root.findall("./Gene")
        genesDict = {}
        for gene in genes:
            geneID = gene.attrib['frameid']
            geneName = gene.findall("common-name")[0].text
            geneStrand= gene.findall("transcription-direction")[0].text
            #print geneStrand
        #print "Gene: " + geneName + ", GeneID: " + geneID
        #page = requests.get("http://biocyc.org/gene?orgid=ECOLI&id=EG11879")
            genesDict[geneID] = [geneName, geneStrand]
        return genesDict
# this function pulls the promoterIDs associated with the geneID passed into it
    def getPromoter(self, geneID):
        promoterDict = {}
        resp  = requests.get("http://ecocyc.org/apixml?fn=transcription-units-of-gene&id=ECOLI:{}&detail=full".format(geneID))
        root=ET.fromstring(resp.text)
    #print(resp.text)
        promoters=root.findall("./Transcription-Unit/component/Promoter")
        for promoter in promoters:
            pID = promoter.attrib['frameid']
            absPositPlusOne = promoter.find('absolute-plus-1-pos').text
            promoterDict[pID]= absPositPlusOne
        return promoterDict
class FastAreader :
    '''
    Class to provide reading of a file containing one or more FASTA
    formatted sequences:
    object instantiation:
    FastAreader(<file name>):
 
    object attributes:
    fname: the initial file name
 
    methods:
    readFasta() : returns header and sequence as strings.
    Author: David Bernick
    Date: April 19, 2013
    '''
    def __init__ (self, fname):
        '''contructor: saves attribute fname '''
        self.fname = fname
 
    def readFasta (self):
        '''
        using filename given in init, returns each included FastA record
        as 2 strings - header and sequence.
        whitespace is removed, no adjustment is made to sequence contents.
        The initial '>' is removed from the header.
        '''
        header = ''
        sequence = ''
        
        with open(self.fname) as fileH:
            # initialize return containers
            header = ''
            sequence = ''
 
            # skip to first fasta header
            line = fileH.readline()
            while not line.startswith('>') :
                line = fileH.readline()
            header = line[1:].rstrip()
 
            # header is saved, get the rest of the sequence
            # up until the next header is found
            # then yield the results and wait for the next call.
            # next call will resume at the yield point
            # which is where we have the next header
            for line in fileH:
                if line.startswith ('>'):
                    yield header,sequence
                    header = line[1:].rstrip()
                    sequence = ''
                else :
                    sequence += ''.join(line.rstrip().split()).upper()
        # final header and sequence will be seen with an end of file
        # with clause will terminate, so we do the final yield of the data
        yield header,sequence
        

myOrg = organismBreadBoard("ECOLI")
myReader = FastAreader ('ecoli_v3_u00096_3.fa');

for head, seq in myReader.readFasta() :
    #print (head,seq)
    targetHead=head
    targetSeq= seq



    
allPathWays = myOrg.pathwayFromCompound("HIS")

for pathway in allPathWays:
    operonDirection=""
    print allPathWays[pathway]
    userPath = raw_input("Do you want to knockdown this function? yes or no.")
    if userPath == "yes":
        genes = myOrg.genesFromPathway(pathway)
        operonPromoterDict={}
        for gene in genes:
            operonDirection=genes[gene][1]
            #print genes[gene], genes[gene][1], gene
            #print genes[gene][0]
            promoters_gene=myOrg.getPromoter(gene)
            dwnStrmPromo_gene=0
            for promoter in promoters_gene:
                if promoters_gene[max(promoters_gene, key=promoters_gene.get)]>dwnStrmPromo_gene:
                    dwnStrmPromo_gene = promoter
                    dwnStrmPromoPos_gene = promoters_gene[max(promoters_gene, key=promoters_gene.get)]
                else:
                    continue
            operonPromoterDict[dwnStrmPromo_gene]= dwnStrmPromoPos_gene
        #print operonPromoterDict
        dwnStrmPromoPos_operon=0
        dwnStrmPromoID_operon=""
        for promoter_operon in operonPromoterDict:
            #print operonPromoterDict[promoter_operon]
            if int(operonPromoterDict[promoter_operon]) > dwnStrmPromoPos_operon:
                dwnStrmPromoPos_operon = int(operonPromoterDict[promoter_operon])
                dwnStrmPromoID_operon = promoter_operon
            else:
                continue
            #print operonPromoterDict[gene][1]
        #print dwnStrmPromoID_operon, dwnStrmPromoPos_operon
        #print max(operonPromoterDict, key=operonPromoterDict.get)
        break
    elif userPath == "no":
        continue
    else:
        print "please enter a valid response. yes or no"
        continue
if operonDirection == "-":
    targetPromoSeq = targetSeq[dwnStrmPromoPos_operon:(dwnStrmPromoPos_operon+50)]
else:
    targetPromoSeq = targetSeq[(dwnStrmPromoPos_operon-50):dwnStrmPromoPos_operon]
print dwnStrmPromoID_operon
print targetPromoSeq


ImportError: No module named biopython

In [ ]:
from IPython.display import display, Image
import xml.etree.ElementTree as ET
import requests
import pythoncyc as pc
from HTMLParser import HTMLParser
from lxml import html
#import biopython

# this function finds all pathways associated with the molecule of interest. needs work on front end to 
# allow user to define G.O. function associated with molecule.
def pathwayFromCompound(compound):
    pathIDs = []
    #request pathway xml from ecocyc server
    resp = requests.get("http://ecocyc.org/apixml?fn=pathways-of-compound&id=ECOLI:{}".format(compound))
    #builds element tree from the response xml text file
    root = ET.fromstring(resp.text)
    #pulls all elements associated with Pathway tag
    pathways = root.findall("./Pathway")
    #pulls all pathway IDs
    for pathway in pathways:
        pathID = pathway.attrib['frameid']
        print (pathID)
        print "Pathway: ", pathway.findall("common-name")[0].text
        #instead of this we can present a tree of the biokernel
        userPath = raw_input("Do you want to knockdown this function? yes or no.")
        if userPath == "yes":
            genesFromPathway(pathID)
            break
        else:
            continue

# this function pulls all geneIDs associated with the pathwayID passed into it
def genesFromPathway(pathwayID):
    resp  = requests.get("http://ecocyc.org/apixml?fn=genes-of-pathway&id=ECOLI:{}".format(pathwayID))
    root = ET.fromstring(resp.text)
    #print(resp.text)
    genes = root.findall("./Gene")
    
    for gene in genes:
        geneID = gene.attrib['frameid']
        geneName = gene.findall("common-name")[0].text
        print "Gene: " + geneName + ", GeneID: " + geneID
        page = requests.get("http://biocyc.org/gene?orgid=ECOLI&id=EG11879")
        #print(page.content)
        #tree = html.fromstring(page.content)
        #mapPost = tree.find("./Map Position")
        #print(mapPost)
        #root1 = ET.fromstring(resp1.text)
        
        getPromoter(geneID)

# this function pulls the promoterIDs associated with the geneID passed into it
def getPromoter(geneID):
    resp  = requests.get("http://ecocyc.org/apixml?fn=transcription-units-of-gene&id=ECOLI:{}".format(geneID))
    root=ET.fromstring(resp.text)
    #print(resp.text)
    promoters=root.findall("./Transcription-Unit/component/Promoter")
    #print promoters
    for promoter in promoters:
        if promoter == "'NoneType'":
            print "NoneType"
        else:
            pID = promoter.attrib['frameid']
            absPositPlusOne = promoter.find('absolute-plus-1-pos').text
        #regulon = ecoli.containing_chromosome(pID)
            print "promoter: ",pID
            print(absPositPlusOne)
        #print(regulon)

#ecoli = pc.select_organism("ECOLI")
pathwayFromCompound("HIS")
#genesFromPathway("HISTSYN-PWY")


HISTSYN-PWY
Pathway:  L-histidine biosynthesis


In [ ]:
from IPython.display import display, Image
import xml.etree.ElementTree as ET
import requests
import pythoncyc as pc
from HTMLParser import HTMLParser
from lxml import html
#import biopython

# this function finds all pathways associated with the molecule of interest. needs work on front end to 
# allow user to define G.O. function associated with molecule.
def pathwayFromCompound(compound):
    pathWays = {}
    #request pathway xml from ecocyc server
    resp = requests.get("http://ecocyc.org/apixml?fn=pathways-of-compound&id=ECOLI:{}".format(compound))
    #builds element tree from the response xml text file
    root = ET.fromstring(resp.text)
    #pulls all elements associated with Pathway tag
    pathways = root.findall("./Pathway")
    #pulls all pathway IDs
    for pathway in pathways:
        pathID = pathway.attrib['frameid']
        pathWays[pathID] = pathway.findall("common-name")[0].text
        #print (pathWays)
    #return dictionary with pathway ID: common name as key:value pairs
    #print pathWays
    return pathWays

# this function pulls all geneIDs associated with the pathwayID passed into it
def genesFromPathway(pathwayID):
    resp  = requests.get("http://ecocyc.org/apixml?fn=genes-of-pathway&id=ECOLI:{}&detail=full".format(pathwayID))
    root = ET.fromstring(resp.text)
    #print(resp.text)
    genes = root.findall("./Gene")
    genesDict = {}
    for gene in genes:
        geneID = gene.attrib['frameid']
        geneName = gene.findall("common-name")[0].text
        geneStrand= gene.findall("transcription-direction")[0].text
            #print geneStrand
        #print "Gene: " + geneName + ", GeneID: " + geneID
        #page = requests.get("http://biocyc.org/gene?orgid=ECOLI&id=EG11879")
        genesDict[geneID] = [geneName, geneStrand]
    return genesDict
# this function pulls the promoterIDs associated with the geneID passed into it
def getPromoter(geneID):
    promoterDict = {}
    resp  = requests.get("http://ecocyc.org/apixml?fn=transcription-units-of-gene&id=ECOLI:{}&detail=full".format(geneID))
    root=ET.fromstring(resp.text)
    #print(resp.text)
    promoters=root.findall("./Transcription-Unit/component/Promoter")
    #transcripts=root.findall("./Transcription-Unit")
    #for TU in transcripts:
        #TUID=TU.attrib['frameid']
        #print TUID
        #resp  = requests.get("http://ecocyc.org/apixml?fn=transcription-unit-genes&id=ECOLI:{}&detail=full".format(TUID))
        #root = ET.fromstring(resp.text)
        #print resp.text
        #resp  = requests.get("http://ecocyc.org/apixml?fn=transcription-unit-promoter&id=ECOLI:{}".format(TUID))
        #root = ET.fromstring(resp.text)
        #print resp.text
    #print promoters
    for promoter in promoters:
        pID = promoter.attrib['frameid']
        absPositPlusOne = promoter.find('absolute-plus-1-pos').text
        #regulon = ecoli.containing_chromosome(pID)
        #print "promoter: ",pID
        promoterDict[pID]= absPositPlusOne
        #print(regulon)
    return promoterDict

        
#EG10444
#TU0-6650
#ecoli = pc.select_organism("ECOLI")
allPathways = pathwayFromCompound("HIS")
#genesFromPathway("HISTSYN-PWY")

#getPromoter('EG10444')

for pathway in allPathways:
    print allPathways[pathway]
    userPath = raw_input("Do you want to knockdown this function? yes or no.")
    if userPath == "yes":
        genes = genesFromPathway(pathway)
        for gene in genes:
            print genes[gene], gene
            promoters=getPromoter(gene)
        #print promoters
        break
    else:
        continue

In [51]:
from IPython.display import display, Image
import xml.etree.ElementTree as ET
import requests
import pythoncyc as pc
from HTMLParser import HTMLParser
from lxml import html
#import biopython

# this function finds all pathways associated with the molecule of interest. needs work on front end to 
# allow user to define G.O. function associated with molecule.
class organismBreadBoard:
    def __init__(self, name):
        self.name = name
    def pathwayFromCompound(self, compound):
        self.compound = compound
        pathWays = {}
    #request pathway xml from ecocyc server
        resp = requests.get("http://ecocyc.org/apixml?fn=pathways-of-compound&id=ECOLI:{}".format(self.compound))
    #builds element tree from the response xml text file
        root = ET.fromstring(resp.text)
    #pulls all elements associated with Pathway tag
        pathways = root.findall("./Pathway")
    #pulls all pathway IDs
        for pathway in pathways:
            pathID = pathway.attrib['frameid']
            pathWays[pathID] = pathway.findall("common-name")[0].text
        #print (pathWays)
    #return dictionary with pathway ID: common name as key:value pairs
    #print pathWays
        return pathWays

# this function pulls all geneIDs associated with the pathwayID passed into it
    def genesFromPathway(self, pathwayID):
        resp  = requests.get("http://ecocyc.org/apixml?fn=genes-of-pathway&id=ECOLI:{}&detail=full".format(pathwayID))
        root = ET.fromstring(resp.text)
    #print(resp.text)
        genes = root.findall("./Gene")
        genesDict = {}
        for gene in genes:
            geneID = gene.attrib['frameid']
            geneName = gene.findall("common-name")[0].text
            geneStrand= gene.findall("transcription-direction")[0].text
            #print geneStrand
        #print "Gene: " + geneName + ", GeneID: " + geneID
        #page = requests.get("http://biocyc.org/gene?orgid=ECOLI&id=EG11879")
            genesDict[geneID] = [geneName, geneStrand]
        return genesDict
# this function pulls the promoterIDs associated with the geneID passed into it
    def getPromoter(self, geneID):
        promoterDict = {}
        resp  = requests.get("http://ecocyc.org/apixml?fn=transcription-units-of-gene&id=ECOLI:{}&detail=full".format(geneID))
        root=ET.fromstring(resp.text)
    #print(resp.text)
        promoters=root.findall("./Transcription-Unit/component/Promoter")
        for promoter in promoters:
            pID = promoter.attrib['frameid']
            absPositPlusOne = promoter.find('absolute-plus-1-pos').text
            promoterDict[pID]= absPositPlusOne
        return promoterDict

        
#EG10444
#TU0-6650
#ecoli = pc.select_organism("ECOLI")
#allPathWays=pathwayFromCompound("HIS").pathwayFromCompound()
myOrg = organismBreadBoard("ECOLI")
allPathWays = myOrg.pathwayFromCompound("HIS")
print type(allPathWays)
#genesFromPathway("HISTSYN-PWY")

#getPromoter('EG10444')

for pathway in allPathWays:
    print allPathWays[pathway]
    userPath = raw_input("Do you want to knockdown this function? yes or no.")
    if userPath == "yes":
        genes = myOrg.genesFromPathway(pathway)
        operonPromoterDict={}
        for gene in genes:
            print genes[gene], genes[gene][1], gene
            #print genes[gene][0]
            promoters_gene=myOrg.getPromoter(gene)
            dwnStrmPromo_gene=0
            for promoter in promoters_gene:
                if promoters_gene[max(promoters_gene, key=promoters_gene.get)]>dwnStrmPromo_gene:
                    dwnStrmPromo_gene = promoter
                    dwnStrmPromoPos_gene = promoters_gene[max(promoters_gene, key=promoters_gene.get)]
                else:
                    continue
            operonPromoterDict[dwnStrmPromo_gene]= dwnStrmPromoPos_gene
        #print operonPromoterDict
        dwnStrmPromoPos_operon=0
        dwnStrmPromoID_operon=""
        for promoter_operon in operonPromoterDict:
            if operonPromoterDict[promoter_operon] > dwnStrmPromoPos_operon:
                dwnStrmPromoPos_operon = operonPromoterDict[promoter_operon]
                dwnStrmPromoID_operon = promoter_operon
            else:
                continue
            #print operonPromoterDict[gene][1]
        print dwnStrmPromoID_operon, dwnStrmPromoPos_operon
        #print max(operonPromoterDict, key=operonPromoterDict.get)
        break
    else:
        continue

<type 'dict'>
tRNA charging
Do you want to knockdown this function? yes or no.no
superpathway of histidine, purine, and pyrimidine biosynthesis
Do you want to knockdown this function? yes or no.no
L-histidine biosynthesis
Do you want to knockdown this function? yes or no.yes
['hisI', '+'] + EG10451
['hisH', '+'] + EG10450
['hisC', '+'] + EG10446
['hisD', '+'] + EG10447
['hisA', '+'] + EG10444
['hisB', '+'] + EG10445
['hisF', '+'] + EG10448
['hisG', '+'] + EG10449
PM00398 2093360


In [1]:
class FastAreader :
    '''
    Class to provide reading of a file containing one or more FASTA
    formatted sequences:
    object instantiation:
    FastAreader(<file name>):
 
    object attributes:
    fname: the initial file name
 
    methods:
    readFasta() : returns header and sequence as strings.
    Author: David Bernick
    Date: April 19, 2013
    '''
    def __init__ (self, fname):
        '''contructor: saves attribute fname '''
        self.fname = fname
 
    def readFasta (self):
        '''
        using filename given in init, returns each included FastA record
        as 2 strings - header and sequence.
        whitespace is removed, no adjustment is made to sequence contents.
        The initial '>' is removed from the header.
        '''
        header = ''
        sequence = ''
        
        with open(self.fname) as fileH:
            # initialize return containers
            header = ''
            sequence = ''
 
            # skip to first fasta header
            line = fileH.readline()
            while not line.startswith('>') :
                line = fileH.readline()
            header = line[1:].rstrip()
 
            # header is saved, get the rest of the sequence
            # up until the next header is found
            # then yield the results and wait for the next call.
            # next call will resume at the yield point
            # which is where we have the next header
            for line in fileH:
                if line.startswith ('>'):
                    yield header,sequence
                    header = line[1:].rstrip()
                    sequence = ''
                else :
                    sequence += ''.join(line.rstrip().split()).upper()
        # final header and sequence will be seen with an end of file
        # with clause will terminate, so we do the final yield of the data
        yield header,sequence
 
# presumed object instantiation and example usage
#myReader = FastAreader ('ecoli_v3_u00096_3.fa');
for head, seq in myReader.readFasta() :
    print (head,seq)


NameError: name 'myReader' is not defined

In [ ]:
#for python 3.5
from IPython.display import display, Image
import xml.etree.ElementTree as ET
import requests
#import pythoncyc as pc
#from HTMLParser import HTMLParser
#from lxml import html
import biopython

# this function finds all pathways associated with the molecule of interest. needs work on front end to 
# allow user to define G.O. function associated with molecule.
class organismBreadBoard:
    def __init__(self, name):
        self.name = name
    def pathwayFromCompound(self, compound):
        self.compound = compound
        pathWays = {}
    #request pathway xml from ecocyc server
        resp = requests.get("http://ecocyc.org/apixml?fn=pathways-of-compound&id=ECOLI:{}".format(self.compound))
    #builds element tree from the response xml text file
        root = ET.fromstring(resp.text)
    #pulls all elements associated with Pathway tag
        pathways = root.findall("./Pathway")
    #pulls all pathway IDs
        for pathway in pathways:
            pathID = pathway.attrib['frameid']
            pathWays[pathID] = pathway.findall("common-name")[0].text
        #print (pathWays)
    #return dictionary with pathway ID: common name as key:value pairs
    #print pathWays
        return pathWays

# this function pulls all geneIDs associated with the pathwayID passed into it
    def genesFromPathway(self, pathwayID):
        resp  = requests.get("http://ecocyc.org/apixml?fn=genes-of-pathway&id=ECOLI:{}&detail=full".format(pathwayID))
        root = ET.fromstring(resp.text)
    #print(resp.text)
        genes = root.findall("./Gene")
        genesDict = {}
        for gene in genes:
            geneID = gene.attrib['frameid']
            geneName = gene.findall("common-name")[0].text
            geneStrand= gene.findall("transcription-direction")[0].text
            #print geneStrand
        #print "Gene: " + geneName + ", GeneID: " + geneID
        #page = requests.get("http://biocyc.org/gene?orgid=ECOLI&id=EG11879")
            genesDict[geneID] = [geneName, geneStrand]
        return genesDict
# this function pulls the promoterIDs associated with the geneID passed into it
    def getPromoter(self, geneID):
        promoterDict = {}
        resp  = requests.get("http://ecocyc.org/apixml?fn=transcription-units-of-gene&id=ECOLI:{}&detail=full".format(geneID))
        root=ET.fromstring(resp.text)
    #print(resp.text)
        promoters=root.findall("./Transcription-Unit/component/Promoter")
        for promoter in promoters:
            pID = promoter.attrib['frameid']
            absPositPlusOne = promoter.find('absolute-plus-1-pos').text
            promoterDict[pID]= absPositPlusOne
        return promoterDict
class FastAreader :
    '''
    Class to provide reading of a file containing one or more FASTA
    formatted sequences:
    object instantiation:
    FastAreader(<file name>):
 
    object attributes:
    fname: the initial file name
 
    methods:
    readFasta() : returns header and sequence as strings.
    Author: David Bernick
    Date: April 19, 2013
    '''
    def __init__ (self, fname):
        '''contructor: saves attribute fname '''
        self.fname = fname
 
    def readFasta (self):
        '''
        using filename given in init, returns each included FastA record
        as 2 strings - header and sequence.
        whitespace is removed, no adjustment is made to sequence contents.
        The initial '>' is removed from the header.
        '''
        header = ''
        sequence = ''
        
        with open(self.fname) as fileH:
            # initialize return containers
            header = ''
            sequence = ''
 
            # skip to first fasta header
            line = fileH.readline()
            while not line.startswith('>') :
                line = fileH.readline()
            header = line[1:].rstrip()
 
            # header is saved, get the rest of the sequence
            # up until the next header is found
            # then yield the results and wait for the next call.
            # next call will resume at the yield point
            # which is where we have the next header
            for line in fileH:
                if line.startswith ('>'):
                    yield header,sequence
                    header = line[1:].rstrip()
                    sequence = ''
                else :
                    sequence += ''.join(line.rstrip().split()).upper()
        # final header and sequence will be seen with an end of file
        # with clause will terminate, so we do the final yield of the data
        yield header,sequence
        

myOrg = organismBreadBoard("ECOLI")
myReader = FastAreader ('ecoli_v3_u00096_3.fa');

for head, seq in myReader.readFasta() :
    #print (head,seq)
    targetHead=head
    targetSeq= seq



    
allPathWays = myOrg.pathwayFromCompound("HIS")

for pathway in allPathWays:
    operonDirection=""
    print (allPathWays[pathway])
    userPath = raw_input("Do you want to knockdown this function? yes or no.")
    if userPath == "yes":
        genes = myOrg.genesFromPathway(pathway)
        operonPromoterDict={}
        for gene in genes:
            operonDirection=genes[gene][1]
            #print genes[gene], genes[gene][1], gene
            #print genes[gene][0]
            promoters_gene=myOrg.getPromoter(gene)
            dwnStrmPromo_gene=0
            for promoter in promoters_gene:
                if promoters_gene[max(promoters_gene, key=promoters_gene.get)]>dwnStrmPromo_gene:
                    dwnStrmPromo_gene = promoter
                    dwnStrmPromoPos_gene = promoters_gene[max(promoters_gene, key=promoters_gene.get)]
                else:
                    continue
            operonPromoterDict[dwnStrmPromo_gene]= dwnStrmPromoPos_gene
        #print operonPromoterDict
        dwnStrmPromoPos_operon=0
        dwnStrmPromoID_operon=""
        for promoter_operon in operonPromoterDict:
            #print operonPromoterDict[promoter_operon]
            if int(operonPromoterDict[promoter_operon]) > dwnStrmPromoPos_operon:
                dwnStrmPromoPos_operon = int(operonPromoterDict[promoter_operon])
                dwnStrmPromoID_operon = promoter_operon
            else:
                continue
            #print operonPromoterDict[gene][1]
        #print dwnStrmPromoID_operon, dwnStrmPromoPos_operon
        #print max(operonPromoterDict, key=operonPromoterDict.get)
        break
    elif userPath == "no":
        continue
    else:
        print ("please enter a valid response. yes or no")
        continue
if operonDirection == "-":
    targetPromoSeq = targetSeq[(dwnStrmPromoPos_operon-50):dwnStrmPromoPos_operon]
else:
    targetPromoSeq = targetSeq[(dwnStrmPromoPos_operon-50):dwnStrmPromoPos_operon]
print (dwnStrmPromoID_operon, operonDirection)
print (targetPromoSeq)

